In [1]:
!pip install captum
!pip install flask
!pip install flask-compress
!pip install ipywidgets

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from captum.insights import AttributionVisualizer, Batch
from captum.insights.attr_vis.features import ImageFeature

In [3]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [4]:
%matplotlib inline

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.bn0 = nn.BatchNorm2d(3)
        self.conv1 = nn.Conv2d(3, 9, 3)
        self.pool1 = nn.AvgPool2d(4, 4)
        
        self.conv1_bn = nn.BatchNorm2d(9)
        self.conv2 = nn.Conv2d(9, 16, 3)
        self.pool2 = nn.AvgPool2d(4, 4)
        
        self.conv2_bn = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 25, 3)
        self.pool3 = nn.AvgPool2d(4, 4)
        
        self.conv3_bn = nn.BatchNorm2d(25)
        self.conv4 = nn.Conv2d(25, 36, 3)
        self.pool4 = nn.AvgPool2d(2 , 2)
        
        self.fc = nn.Linear(324, 4)

    def forward(self, x):
        x = self.bn0(x)
        x = self.conv1_bn(self.pool1(F.relu(self.conv1(x))))
        x = self.conv2_bn(self.pool2(F.relu(self.conv2(x))))
        x = self.conv3_bn(self.pool3(F.relu(self.conv3(x))))
        x = self.pool4(F.relu(self.conv4(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc(x)
        return x


model = Net()
model.load_state_dict(torch.load("saved_models/kitty/epoch_8_batch_0.pth"))


<All keys matched successfully>

In [6]:
def get_classes():
    classes = [
        "Amsterdam",
        "NYC",
        "Firenca",
        "LasVegas",
    ]
    return classes

In [7]:
def baseline_func(input):
    return input * 0

transform = transforms.Compose([
    transforms.ToTensor(),
])

def formatted_data_iter():
    dataset = torchvision.datasets.ImageFolder(root='dataset/test', transform=transform)
    dataloader = iter(
        torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=1)
    )
    while True:
        images, labels = next(dataloader)
        yield Batch(inputs=images, labels=labels)

In [8]:
visualizer = AttributionVisualizer(
    models=[model],
    score_func=lambda o: torch.nn.functional.softmax(o, 1),
    classes=get_classes(),
    features=[
        ImageFeature(
            "Photo",
            baseline_transforms=[baseline_func],
            input_transforms=[]
        )
    ],
    dataset=formatted_data_iter(),
)

In [9]:
visualizer.render(debug=True)

CaptumInsights(insights_config={'classes': ['Amsterdam', 'NYC', 'Firenca', 'LasVegas'], 'methods': ['Deconvolu…

Output()